# 07 Data Wrangling
__Math 3080: Fundamentals of Data Science__

Reading:
* McKinney, Chapter 7 - Data Cleaning and Preparation
* McKinney, Chapter 8 - Data Wrangling: Join, Combine, and Reshape

Outline:
1. Mapping
2. Sampling
3. Dummy Variables / Indicators
    * Value counts
4. Joining two datasets
5. Pivot tables
6. Groupbys

Other methods discussed in the book that we won't cover here, but are valuable resources:
* Regular Expressions
* String methods and manipulation

-----
We often have two sets of data on the same subject, and both add a good deal of information. Wouldn't it be nice to merge the datasets together? If we could do that, our options for what to do with data would increase significantly. 

Also, what if the data is not quite in the format we want? For example, what if we have a list of observations by date, but we'd like to change that to a table with dates indicating the row and the columns indicate the year?

In this section, we will look at how we can accomplish both of these tasks. It is part of a branch of data science called __data wrangling__.

## 7.1 Mapping
Sometimes, we have a dataset that could use a little more information. Take the following dataset on different kinds of meat:

In [1]:
import numpy as np
import pandas as pd

In [20]:
meat_data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                      "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
meat_data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


We have a variety of different meats. Let's add a little information to indicate what type of animal each meat type comes from. We do this with a technique called __mapping__. This takes the value from one variable of your dataset and looks up a second value based on the first from another list. For example, "bacon" in the food variable would have any entry in the other list that would return the animal "pig".

In [21]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

meat_data['Animal'] = meat_data['food'].map(meat_to_animal)
meat_data

,food,ounces,Animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


## 7.2 Sampling

In [22]:
samples = np.random.permutation(9)
samples

array([8, 1, 0, 3, 6, 5, 2, 4, 7])

In [23]:
meat_data.iloc[samples]

,food,ounces,Animal
8,nova lox,6.0,salmon
1,pulled pork,3.0,pig
0,bacon,4.0,pig
3,pastrami,6.0,cow
6,pastrami,3.0,cow
5,bacon,8.0,pig
2,bacon,12.0,pig
4,corned beef,7.5,cow
7,honey ham,5.0,pig


In [24]:
meat_data.sample(n=4)

,food,ounces,Animal
1,pulled pork,3.0,pig
6,pastrami,3.0,cow
0,bacon,4.0,pig
3,pastrami,6.0,cow


## 7.3 Dummy Variables / Indicators
* Value counts


In [26]:
meat_data

,food,ounces,Animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [25]:
pd.get_dummies(meat_data['Animal'])

,cow,pig,salmon
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,1,0,0
5,0,1,0
6,1,0,0
7,0,1,0
8,0,0,1


If we want to see them together, we can do a join, which we discuss next.

## 7.4 Joining two datasets
We can merge (join) two datasets. When we join datasets, there are 4 methods in which they can be joined:
* Left join (all data in left table is kept, any unmatched data from the right table is dropped)
* Right join (all data in right table is kept, any unmatched data from the left table is dropped)
* Inner join (only data that matches both left and right tables are kept)
* Outer join (all data are kept, whether they match or not)

![Different types of joins](https://d33wubrfki0l68.cloudfront.net/9c12ca9e12ed26a7c5d2aa08e36d2ac4fb593f1e/79980/diagrams/join-outer.png)
* image from *R for Data Science*, Hadley Wickham & Garret Grolemund, 2017.

When we perform a join, the computer looks for a column that matches between the two datasets. If it can't find a match, it will match the indices.

In [36]:
meat_data.join(pd.get_dummies(meat_data['Animal']))

,food,ounces,Animal,cow,pig,salmon
0,bacon,4.0,pig,0,1,0
1,pulled pork,3.0,pig,0,1,0
2,bacon,12.0,pig,0,1,0
3,pastrami,6.0,cow,1,0,0
4,corned beef,7.5,cow,1,0,0
5,bacon,8.0,pig,0,1,0
6,pastrami,3.0,cow,1,0,0
7,honey ham,5.0,pig,0,1,0
8,nova lox,6.0,salmon,0,0,1


Let's take a closer look at the different types of joins.

In [29]:
data_A = pd.DataFrame({'key':[1,2,3,4,5,6,7],
                       'value':[12,13,14,15,16,17,18]})
data_B = pd.DataFrame({'key':[1,3,5,7,9,11,13],
                       'value':[22,23,24,25,26,27,28]})

data_A

,key,value
0,1,12
1,2,13
2,3,14
3,4,15
4,5,16
5,6,17
6,7,18


In [30]:
data_B

,key,value
0,1,22
1,3,23
2,5,24
3,7,25
4,9,26
5,11,27
6,13,28


__Left Join__

In [32]:
data_A.merge(data_B, on='key', how='left')

,key,value_x,value_y
0,1,12,22.0
1,2,13,NaN
2,3,14,23.0
3,4,15,NaN
4,5,16,24.0
5,6,17,NaN
6,7,18,25.0


__Right Join__

In [33]:
data_B.merge(data_A, on='key', how='right')

,key,value_x,value_y
0,1,22.0,12
1,2,NaN,13
2,3,23.0,14
3,4,NaN,15
4,5,24.0,16
5,6,NaN,17
6,7,25.0,18


__Inner Join__

__Outer Join__

## 7.5 Pivot tables


## 7.6 Groupbys